In [24]:
library(tidyverse)
library(gridExtra)
library(kableExtra)
options(kableExtra.html.bsTable = T)

*Adapted from [Emily Robinson](https://hookedondata.org/pokemon-type-combinations/). Data from [robinsones](https://github.com/robinsones/pokemon-chart/blob/master/chart.csv).*

### Introduction

Came across this fairly interesting post recently. Given that there are 18 types of pokemon, some of which are super-effective/not very effective against each other, how do we pick a pokemon team combination that maximises effectiveness against the bulk of other teams? We take in a base dataset showing the effects from each 18*18 attack/defence pair. 0.5 indicates a not very effective attack, 1 indicates a normal attack, and 2 indicates a super effective attack. 


In [37]:
chart <- read.csv('~/Desktop/yjtek.github.io/data/2019-12-16-reproduced-best-pokemon-team/chart.csv')
chart

Attacking,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Normal,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5,1.0
Fire,1,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0,1.0
Water,1,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0,1.0
Electric,1,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0
Grass,1,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5,1.0
Ice,1,0.5,0.5,1.0,2.0,0.5,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0
Fighting,2,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,0.5,0.5,0.5,2.0,0.0,1.0,2.0,2.0,0.5
Poison,1,1.0,1.0,1.0,2.0,1.0,1.0,0.5,0.5,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.0,2.0
Ground,1,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,0.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0,1.0


### Basic analysis

We can already draw some basic aggregate conclusions about the most useful types by maximising the attack dealt and minimising the attack received. Remember that the baseline for both scores is 18 (i.e. you deal normal damage to all other types, and take normal damage from all other types). 

In [38]:
mostUsefulAttack <- data.frame(type = chart$Attacking, `Attack Dealt` = rowSums(chart[,2:ncol(chart)]))
mostUsefulDefence <- data.frame(type = colnames(chart)[2:ncol(chart)], `Attack Received` = colSums(chart[, 2:ncol(chart)]))
mostUseful <- mostUsefulAttack %>% left_join(mostUsefulDefence, by = 'type') %>% `colnames<-`(c('Type', 'Attack Dealt', 'Attack Received'))
mostUseful

Type,Attack Dealt,Attack Received
<chr>,<dbl>,<dbl>
Normal,16.0,18.0
Fire,20.0,18.0
Water,19.5,18.0
Electric,17.5,17.5
Grass,17.5,21.0
Ice,20.0,21.5
Fighting,19.5,19.5
Poison,17.0,17.5
Ground,21.0,19.0


A cursory analysis will already tell us which types are most useful. If we are willing to ignore the distribution of the scores, we can simply find all types where attack dealt exceeds the baseline, and the attack received is below the baseline.

In [40]:
mostUseful %>%
  filter(`Attack Dealt` >= 18 & `Attack Received` <= 18)

Type,Attack Dealt,Attack Received
<chr>,<dbl>,<dbl>
Fire,20.0,18.0
Water,19.5,18.0
Ghost,18.5,17.0
Steel,19.0,15.0
Fairy,19.5,17.5


### Maximising super-effectiveness

As marvellous as that sounds, we clearly don't fight pokemon in some weird aggregated group fight. Pokemon fights are a 1v1 affair, and it follows that the analysis should be conducted at the type pair level. We sharpen the granularity of the analysis to see which types provide the most "super effective" attacks.

In [44]:
chartLong <- chart %>%
  pivot_longer(-Attacking, names_to = 'Defending', values_to = 'Attack Effectiveness') %>%
  mutate(`Attack Effectiveness` = if_else(`Attack Effectiveness` == 2, 1, 0)) %>%
  group_by(Attacking) %>%
  summarise(`Count Attack Super Effective` = sum(`Attack Effectiveness`)) %>%
  arrange(desc(`Count Attack Super Effective`))
chartLong

`summarise()` ungrouping output (override with `.groups` argument)



Attacking,Count Attack Super Effective
<chr>,<dbl>
Fighting,5
Ground,5
Fire,4
Ice,4
Rock,4
Bug,3
Fairy,3
Flying,3
Grass,3


To form the best combination of 6, and lacking some sort of prior about what that combination should be, we rely on good old fashioned brute forcing. Using `combn()`, we get a dataframe where every column is 1 combination. Using the base dataframe, we change the values such that super effective attacks are reflected as 1s in the matrix, and everything else is reflected as 0.

In [62]:
combinations <- combn(18,6) #get all 6 value combinations of seq(1, 18). Basically this is every possible combination of pokemon teams
m <- as.matrix(chart[, -1]) #get matrix of attack details without the column of types
rownames(m) <- chart$Attacking
super_effective_m <- (m == 2) * 1L
super_effective_m #if super-effective, return 1. Otherwise return 0

,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
Normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Fire,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0
Water,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
Electric,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Grass,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
Ice,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0
Fighting,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0
Poison,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
Ground,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0
Flying,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0


Now we have an 18x18 matrix with 1s where attacks are super effective, a 6x18564 matrix of possible combinations of 6 pokemon teams.. To work with this, we define a function that takes in each combination of pokemon types (every column of `combinations`), and sums the number of types the combination is super effective against. We find the types that appear in the most number of "best combinations".

In [88]:
super_effective_nb <- function(indices){
#   for each pokemon group (6-row-subset of super_effective_m), take the column sum for each of the 18 columns. 
#   If colSums > 0, then there is at least 1 of the 6 types that is super effective against the type, so count 
#   this as a 1. Sum all types your group is super effective against.
  sum(colSums(super_effective_m[indices, ]) > 0)
}
super_effective_results <- apply(combinations, 2, super_effective_nb) #find the number of super-effectives for each possible combination
best_combos <- combinations[, super_effective_results == max(super_effective_results)] #10 out of 18564
strongest_teams <- matrix(rownames(super_effective_m)[best_combos], nrow = 6)
strongest_teams %>%
  data.frame() %>%
  mutate(count = 1:6) %>%
  pivot_longer(-count, names_to = 'group', values_to = 'type') %>%
  select(-count) %>%
  group_by(type) %>%
  summarise(count = length(unique(group))) %>%
  arrange(desc(count))

`summarise()` ungrouping output (override with `.groups` argument)



type,count
<chr>,<int>
Ground,10
Fighting,8
Flying,8
Ice,8
Dark,5
Ghost,5
Grass,4
Poison,4
Electric,2


We can now see the pokemon types that are needed for the maximum possible super-effective team combination. (you cannot run away from having a ground type)

### Equilibrium

Let's assume this whole exercise is correct up to this point (ignore distribution of super-effective attacks, super-effective attacks worth 4x normal attacks, etc.). If we assume that this "optimal" response will be played, is the Nash equilibrium to reply with this? For illustration, I will just use the first group identified

In [89]:
# Picking one of 10 the strongest combinations:
strongestTeamIndex <- unique(row(super_effective_m)[rownames(super_effective_m) %in% strongest_teams[,1]])
super_effective_m[,strongestTeamIndex]

,Electric,Ice,Fighting,Ground,Flying,Ghost
Normal,0,0,0,0,0,0
Fire,0,1,0,0,0,0
Water,0,0,0,1,0,0
Electric,0,0,0,0,1,0
Grass,0,0,0,1,0,0
Ice,0,0,0,1,1,0
Fighting,0,1,0,0,0,0
Poison,0,0,0,0,0,0
Ground,1,0,0,0,0,0
Flying,0,0,1,0,0,0


In [33]:
super_effective_nb_subset <- function(indices){
    sum(colSums(super_effective_m[indices, strongestTeamIndex]) > 0)
}
super_effective_results_subset <- apply(combinations, 2, super_effective_nb_subset)
best_combos_subset <- combinations[, super_effective_results_subset == max(super_effective_results_subset)]
dim(best_combos_subset)

[1]   6 396

Clearly, when you have a fixed team of only 6 to counter, there are a lot more possible "optimal" combinations. The point is to see if you have the optimal 10 among the 396 combinations identified.

In [34]:
subset <- t(best_combos_subset) %>%
  as.data.frame() %>%
  unite('string', V1:V6)
optimal <- t(best_combos) %>%
  as.data.frame() %>%
  unite('string', V1:V6)
  
optimal$string %in% subset$string

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

All 10 appear in the subset! Which suggests that, ceteris paribus, it is always better to choose among the 10 optimal strategies identified above that maximise super effective attacks when responding to a `c(Electric, Ice, Fighting, Ground, Flying, Ghost)` team. 

<b id="f1">1</b> $\frac{18!}{12! \cdot 6!} = 18564$ [↩](#a1)